In [ ]:
%run -p train_cnf.py --data mnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 8000 --test_batch_size 8000

/tancode/repos/tan-ffjord/train_cnf.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.odenvp as odenvp
import lib.multiscale_parallel as multiscale_parallel

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
parser = argparse.ArgumentParser("Continuous Normalizing Flow")
parser.add_argument("--data", choices=["mnist", "svhn", "cifar10", 'lsun_church'], type=str, default="mnist")


do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 0007 | Time 28.6345(59.2484) | Bit/dim 30.0261(31.2093) | Steps 410(410.00) | Grad Norm 195.4694(204.0550) | Total Time 10.00(10.00)
Iter 0008 | Time 26.6953(58.2718) | Bit/dim 29.7526(31.1656) | Steps 410(410.00) | Grad Norm 192.6299(203.7122) | Total Time 10.00(10.00)
Iter 0009 | Time 26.8341(57.3287) | Bit/dim 29.2970(31.1095) | Steps 410(410.00) | Grad Norm 188.7514(203.2634) | Total Time 10.00(10.00)
Iter 0010 | Time 26.6574(56.4086) | Bit/dim 28.8587(31.0420) | Steps 410(410.00) | Grad Norm 185.2241(202.7222) | Total Time 10.00(10.00)
Iter 0011 | Time 27.4943(55.5411) | Bit/dim 28.3464(30.9611) | Steps 410(410.00) | Grad Norm 181.6699(202.0907) | Total Time 10.00(10.00)
Iter 0012 | Time 27.6220(54.7036) | Bit/dim 27.9550(30.8710) | Steps 410(410.00) | Grad Norm 178.0178(201.3685) | Total Time 10.00(10.00)
Iter 0013 | Time 27.3632(53.8833) | Bit/dim 27.4422(30.7681) | Steps 410(410.00) | Grad Norm 173.7989(200.5414) | Tota

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 0014 | Time 26.3122(53.0562) | Bit/dim 26.9219(30.6527) | Steps 410(410.00) | Grad Norm 169.6312(199.6141) | Total Time 10.00(10.00)
Iter 0015 | Time 26.6915(52.2653) | Bit/dim 26.3128(30.5225) | Steps 410(410.00) | Grad Norm 165.0119(198.5760) | Total Time 10.00(10.00)
Iter 0016 | Time 26.7406(51.4995) | Bit/dim 25.7874(30.3805) | Steps 410(410.00) | Grad Norm 160.4112(197.4311) | Total Time 10.00(10.00)
Iter 0017 | Time 26.5418(50.7508) | Bit/dim 25.3163(30.2285) | Steps 410(410.00) | Grad Norm 154.4341(196.1412) | Total Time 10.00(10.00)
Iter 0018 | Time 26.6846(50.0288) | Bit/dim 24.6536(30.0613) | Steps 410(410.00) | Grad Norm 149.7416(194.7492) | Total Time 10.00(10.00)
Iter 0019 | Time 27.3518(49.3485) | Bit/dim 24.0571(29.8812) | Steps 410(410.00) | Grad Norm 144.4533(193.2403) | Total Time 10.00(10.00)
Iter 0020 | Time 26.9458(48.6764) | Bit/dim 23.5203(29.6903) | Steps 410(410.00) | Grad Norm 138.3683(191.5941) | Tota

...

===> Using batch size 8000. Total 7 iterations/epoch.
Iter 1190 | Time 54.9556(54.3819) | Bit/dim 1.3560(1.3768) | Steps 596(596.96) | Grad Norm 8.1618(9.8058) | Total Time 10.00(10.00)
Iter 1191 | Time 55.6744(54.4207) | Bit/dim 1.3379(1.3757) | Steps 602(597.12) | Grad Norm 4.6324(9.6506) | Total Time 10.00(10.00)
Iter 1192 | Time 57.2120(54.5044) | Bit/dim 1.3512(1.3749) | Steps 602(597.26) | Grad Norm 3.4643(9.4650) | Total Time 10.00(10.00)
Iter 1193 | Time 56.4293(54.5622) | Bit/dim 1.3490(1.3741) | Steps 596(597.22) | Grad Norm 6.3099(9.3703) | Total Time 10.00(10.00)
Iter 1194 | Time 56.8145(54.6297) | Bit/dim 1.3453(1.3733) | Steps 602(597.37) | Grad Norm 6.8700(9.2953) | Total Time 10.00(10.00)
Iter 1195 | Time 53.7514(54.6034) | Bit/dim 1.3462(1.3725) | Steps 596(597.33) | Grad Norm 5.2604(9.1743) | Total Time 10.00(10.00)
Iter 1196 | Time 56.4037(54.6574) | Bit/dim 1.3429(1.3716) | Steps 596(597.29) | Grad Norm 2.8372(8.9841) | Total Time 10.00(10.00)
validating...
Epoch 01

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 1197 | Time 55.9160(54.6952) | Bit/dim 1.3385(1.3706) | Steps 596(597.25) | Grad Norm 4.9279(8.8625) | Total Time 10.00(10.00)
Iter 1198 | Time 53.6777(54.6646) | Bit/dim 1.3441(1.3698) | Steps 596(597.21) | Grad Norm 7.6156(8.8251) | Total Time 10.00(10.00)
Iter 1199 | Time 57.9238(54.7624) | Bit/dim 1.3375(1.3688) | Steps 602(597.35) | Grad Norm 11.7199(8.9119) | Total Time 10.00(10.00)
Iter 1200 | Time 53.6977(54.7305) | Bit/dim 1.3662(1.3687) | Steps 596(597.31) | Grad Norm 14.7610(9.0874) | Total Time 10.00(10.00)
Iter 1201 | Time 52.3986(54.6605) | Bit/dim 1.3466(1.3681) | Steps 602(597.45) | Grad Norm 14.1742(9.2400) | Total Time 10.00(10.00)
Iter 1202 | Time 53.7946(54.6345) | Bit/dim 1.3492(1.3675) | Steps 596(597.41) | Grad Norm 10.2849(9.2713) | Total Time 10.00(10.00)
Iter 1203 | Time 57.5326(54.7215) | Bit/dim 1.3382(1.3666) | Steps 602(597.55) | Grad Norm 5.8935(9.1700) | Total Time 10.00(10.00)
validating...
Epoc

do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate
do _generate


===> Using batch size 8000. Total 7 iterations/epoch.
Iter 1204 | Time 59.1114(54.8532) | Bit/dim 1.3374(1.3658) | Steps 596(597.50) | Grad Norm 2.5433(8.9712) | Total Time 10.00(10.00)
Iter 1205 | Time 53.3230(54.8073) | Bit/dim 1.3269(1.3646) | Steps 596(597.46) | Grad Norm 4.4628(8.8359) | Total Time 10.00(10.00)
Iter 1206 | Time 52.4858(54.7376) | Bit/dim 1.3337(1.3637) | Steps 596(597.41) | Grad Norm 7.2193(8.7874) | Total Time 10.00(10.00)
Iter 1207 | Time 51.7985(54.6495) | Bit/dim 1.3432(1.3630) | Steps 596(597.37) | Grad Norm 8.5628(8.7807) | Total Time 10.00(10.00)
